In [ ]:
import pandas as pd
import numpy as np
import warnings
import locale
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f'{mes_nombre}{str(año)[2:]}' # ENE24
fecha = f'{año}{str(mes).zfill(2)}' # 202401

nombre_base = f'base_asignacion_20241202.xlsx' # Modificar
asignacion_path = f'bases/asignacion/{fecha}/{nombre_base}'
asignacion_pagos_path = f'bases/asignacion/{fecha}/base_asignacion_{mes_año}.xlsx'
asignacion_camp_path = f'bases/asignacion/{fecha}/base_asignacion_{mes_año}_CAMP.xlsx'
stock_judicial_path = f'bases/asignacion/{fecha}/StockJudicial.xlsx'
stock_castigo_path = f'bases/asignacion/{fecha}/StockCastigos.xlsx'

print(asignacion_path)
print(asignacion_pagos_path)
print(asignacion_camp_path)
print(stock_judicial_path)
print(stock_castigo_path)

In [60]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [61]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion?', icon='warning')

if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path, sheet_name='Hoja1', dtype={'codigo': str, 'contrato': str, 'cont_18': str})
    df_asignacion.columns = clean_columns(df_asignacion.columns)
    print('Base Asignación:', df_asignacion.shape)

root.destroy()

In [62]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', 'Separar base asignación por agencia?', icon='warning')

if result == 'yes':
    def separar_asignacion(df: pd.DataFrame, agencia: str) -> pd.DataFrame:
        df_asignacion_agencia = df[df['AGENCIA'] == agencia]
        print(f'Base de Asignacion: {agencia} - {df_asignacion_agencia.shape}')
        print('--------------------------------------------')
        print(df_asignacion_agencia[['AGENCIA', 'CLAVE']].value_counts(dropna=False))
        print('\n')
        
        return df_asignacion_agencia
    
    df_asignacion_rj = separar_asignacion(df_asignacion, 'ASESCOM RJ')
    df_asignacion_clasa = separar_asignacion(df_asignacion, 'CLASA MORA')
    df_asignacion_mornese = separar_asignacion(df_asignacion, 'MORNESE MORA')

root.destroy()

In [ ]:
df_asignacion_test = df_asignacion.copy()
print(df_asignacion_test.columns)
print('Base Asignacion:', df_asignacion_test.shape)

In [ ]:
df_asignacion_test[['AMBITO', 'PRODUCTO']].value_counts(dropna=False).sort_index()

In [65]:
def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA_CORRECTA' in df.columns:
        df['AGENCIA_CORRECTA'].replace(' ', '', inplace=True)
        
        if 'AGENCIA' not in df.columns:
            df.rename(columns={'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)
        else:
            df['AGENCIA'] = np.where(
                df['AGENCIA'].notna() & (df['AGENCIA'].astype(str).str.len() > 0),
                df['AGENCIA'],
                df['AGENCIA_CORRECTA']
            )
            df.drop(columns=['AGENCIA_CORRECTA'], inplace=True)
    
    if 'CONTRATO' in df.columns:
        df.drop(columns=['CONTRATO'], inplace=True)
    
    if 'ID_VTA' not in df.columns:
        df['ID_VTA'] = np.nan
    
    if 'CAMPAÑA' not in df.columns:
        df['CAMPAÑA'] = np.nan
    
    df.rename(columns={'CODIGO': 'CC', 'CONT_18': 'CONTRATO', 'NOMBRE': 'NOMBRE_CLIENTE', 'CLAVE': 'CARTERA'}, inplace=True)
    
    return df

def update_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = rename_columns(df)
    
    df['TIPO_FONDO'].fillna('NULL', inplace=True)
    df['ID_VTA'].fillna('NULL', inplace=True)
    df['ID_VTA'].replace('-', 'NULL', inplace=True)
    
    df['CONTRATO'] = df['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    df['CC'] = df['CC'].astype('Int64').astype(str).str.zfill(8)
    
    conteo_cc = df['CC'].value_counts()
    df['FLAG'] = df['CC'].map(conteo_cc)
    
    df = df[df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA', 'SIN AGENCIA'])]
    
    df['TIPO_FONDO'] = df['TIPO_FONDO'].str.strip()
    
    if 'REACTIVA_KST' in df['TIPO_FONDO'].unique():
        df['TIPO_FONDO'] = df['TIPO_FONDO'].replace({'REACTIVA_KST': 'REACTIVA'})
    
    df.sort_values(by=['CC', 'CONTRATO', 'CAPITAL'], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df[cols]

In [ ]:
cols_required = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'FLAG', 'OFICINA', 'TERRITORIO', 'ID_VTA', 'AMBITO', 'PRODUCTO']

df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)
df_asignacion_test = update_columns(df_asignacion_test, cols_required)
print(df_asignacion_test.shape)

cols_asig_pagos = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'FLAG']
df_asignacion_pagos = df_asignacion_test[cols_asig_pagos]

cols_asig_camp = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'CARTERA', 'TIPO_FONDO', 'TIPO_CARTERA', 'AGENCIA', 'OFICINA', 'TERRITORIO', 'ID_VTA']
df_asignacion_camp = df_asignacion_test[cols_asig_camp]

In [ ]:
df_asignacion_test['PRODUCTO'].value_counts(dropna=False)

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', 'Cargar stocks con base asignación?', icon='warning')

if result == 'yes':
    def cartera_extrajudicial(df: pd.DataFrame) -> pd.DataFrame:
        df['TIPO_CARTERA'] = np.where(
            (df['CARTERA'] == 'EXJ'), 'UNSECURED', df['TIPO_CARTERA']
        )
        print('Asignacion EXJ Actualizada')
        print('--------------------------------------------')
        print(df[['CARTERA', 'TIPO_CARTERA']].value_counts(dropna=False))
        print('\n')
        
        return df
    
    def cartera_judicial(df: pd.DataFrame) -> pd.DataFrame:
        df_cartera_judicial = pd.read_excel(stock_judicial_path)
        cols_judicial = ['CONTRATO', 'TIPO_CARTERA']
        df_cartera_judicial = df_cartera_judicial[cols_judicial]
        df_cartera_judicial['CONTRATO'] = df_cartera_judicial['CONTRATO'].apply(lambda x: x[-18:] if pd.notna(x) else x)
        
        print('Stock Judicial')
        print('--------------------------------------------')
        print(df_cartera_judicial['TIPO_CARTERA'].value_counts(dropna=False))
        print('\n')
        
        df_bc = df[df['CARTERA'] == 'BC']
        df_bc = df_bc.merge(df_cartera_judicial, on='CONTRATO', how='left', suffixes=('', '_judicial'))
        
        tipo_cartera_mapping = df_bc.set_index('CONTRATO')['TIPO_CARTERA_judicial'].to_dict()
        df['TIPO_CARTERA'] = np.where(
            df['CARTERA'] == 'BC',
            df['CONTRATO'].map(tipo_cartera_mapping),
            df['TIPO_CARTERA']
        )
        print('Asignacion BC Actualizada')
        print('--------------------------------------------')
        print(df[['CARTERA', 'TIPO_CARTERA']].value_counts(dropna=False))
        print('\n')
        
        return df
    
    def cartera_castigo(df: pd.DataFrame) -> pd.DataFrame:
        df_cartera_castigo = pd.read_excel(stock_castigo_path)
        df_cartera_castigo.drop(columns=['TIPO_CARTERA'], inplace=True)
        df_cartera_castigo.rename(columns={'GRUPO': 'TIPO_CARTERA'}, inplace=True)
        cols_castigo = ['CONTRATO', 'TIPO_CARTERA']
        df_cartera_castigo = df_cartera_castigo[cols_castigo]
        df_cartera_castigo['CONTRATO'] = df_cartera_castigo['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
        
        print('Stock Castigos')
        print('--------------------------------------------')
        print(df_cartera_castigo['TIPO_CARTERA'].value_counts(dropna=False))
        print('\n')
        
        df_kstbc = df[df['CARTERA'] == 'KSTBC']
        df_kstbc = df_kstbc.merge(df_cartera_castigo, on='CONTRATO', how='left', suffixes=('', '_castigo'))
        
        tipo_cartera_mapping = df_kstbc.set_index('CONTRATO')['TIPO_CARTERA_castigo'].to_dict()
        df['TIPO_CARTERA'] = np.where(
            df['CARTERA'] == 'KSTBC',
            df['CONTRATO'].map(tipo_cartera_mapping),
            df['TIPO_CARTERA']
        )
        print('Asignacion KSTBC Actualizada')
        print('--------------------------------------------')
        print(df[['CARTERA', 'TIPO_CARTERA']].value_counts(dropna=False))
        print('\n')
        
        return df
    
    df_asignacion_pagos = cartera_extrajudicial(df_asignacion_pagos)
    df_asignacion_pagos = cartera_judicial(df_asignacion_pagos)
    df_asignacion_pagos = cartera_castigo(df_asignacion_pagos)

root.destroy()

In [ ]:
df_asignacion_pagos[['CARTERA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion para campañas?', icon='warning')

df_asignacion_pagos.to_excel(asignacion_pagos_path, index=False, sheet_name='BASE')
print('Base Asignación:', df_asignacion_pagos.shape)

if result == 'yes':
    df_asignacion_camp.to_excel(asignacion_camp_path, index=False, sheet_name='BASE')
    print('Base Asignación Campañas:', df_asignacion_camp.shape)

root.destroy()